<a href="https://colab.research.google.com/github/jonathanahc/Proyecto/blob/main/ProyectoProgADAP_255497.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Proyecto de Programación para Analítica Descriptiva y Predictiva**
## Nombre del Proyecto: Consumo y Costo del Agua en Nueva York
## Grupo: A
## Autor: Jonathan Adrian Herrera Castro
## Matrícula: 255497
## Fecha: 19/02/2025

# **1. Introducción**

## - Descripción del problema: ¿Qué queremos analizar o predecir?
## - Objetivo del análisis: ¿Qué impacto tendrá este análisis?
## - Fuente de datos: ¿De dónde provienen los datos?


# **2. Captura y Carga de Datos**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
#import re

In [3]:
df = pd.read_csv('/content/drive/MyDrive/ProyectoProgramacion/Water_Consumption_And_Cost__2013_-_Feb_2023__20250216.csv', engine="python")

In [4]:
df.head()

,Development Name,Borough,Account Name,Location,Meter AMR,Meter Scope,TDS #,EDP,RC Code,Funding Source,...,Service End Date,# days,Meter Number,Estimated,Current Charges,Rate Class,Bill Analyzed,Consumption (HCF),Water&Sewer Charges,Other Charges
0,HOWARD AVENUE,BROOKLYN,HOWARD AVENUE,BLD 02,AMR,NaN,339.0,782,K033900,FEDERAL,...,04/23/2020,31.0,E11310572,N,2945.22,Basic Water and Sewer,Yes,285,2945.22,0.0
1,BAISLEY PARK,QUEENS,BAISLEY PARK,BLD 09,AMR,BLD 09,91.0,240,Q009100,FEDERAL,...,01/26/2020,34.0,K13060723,N,196.35,Basic Water and Sewer,Yes,19,196.35,0.0
2,BAISLEY PARK,QUEENS,BAISLEY PARK,BLD 09,AMR,BLD 09,91.0,240,Q009100,FEDERAL,...,02/24/2020,29.0,K13060723,N,258.35,Basic Water and Sewer,Yes,25,258.35,0.0
3,BAISLEY PARK,QUEENS,BAISLEY PARK,BLD 09,AMR,BLD 09,91.0,240,Q009100,FEDERAL,...,03/23/2020,28.0,K13060723,N,217.02,Basic Water and Sewer,Yes,21,217.02,0.0
4,BAISLEY PARK,QUEENS,BAISLEY PARK,BLD 09,AMR,BLD 09,91.0,240,Q009100,FEDERAL,...,04/23/2020,31.0,K13060723,N,103.34,Basic Water and Sewer,Yes,10,103.34,0.0


In [5]:
print(f"Observaciones: {df.shape[0]}, Variables: {df.shape[1]}")

Observaciones: 50315, Variables: 25


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50315 entries, 0 to 50314
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Development Name     50255 non-null  object 
 1   Borough              50315 non-null  object 
 2   Account Name         50315 non-null  object 
 3   Location             49487 non-null  object 
 4   Meter AMR            49805 non-null  object 
 5   Meter Scope          12782 non-null  object 
 6   TDS #                50255 non-null  float64
 7   EDP                  50315 non-null  int64  
 8   RC Code              50315 non-null  object 
 9   Funding Source       50239 non-null  object 
 10  AMP #                50193 non-null  object 
 11  Vendor Name          50315 non-null  object 
 12  UMIS BILL ID         50315 non-null  int64  
 13  Revenue Month        50315 non-null  object 
 14  Service Start Date   50308 non-null  object 
 15  Service End Date     50308 non-null 

In [12]:
df.describe()

,TDS #,EDP,UMIS BILL ID,# days,Current Charges,Consumption (HCF),Water&Sewer Charges,Other Charges
count,50255.000000,50315.000000,5.031500e+04,50308.000000,50315.000000,50315.000000,50315.000000,50315.000000
mean,258.871794,463.205764,6.597390e+06,33.182953,2150.221066,224.291066,2140.327436,9.893631
std,101.467018,185.109210,2.671463e+06,53.181840,5623.852543,2020.962056,5619.812518,607.714232
min,1.000000,200.000000,2.576038e+06,-335.000000,-1748.510000,0.000000,0.000000,-67726.830000
25%,209.000000,325.000000,4.132859e+06,29.000000,46.460000,5.000000,46.360000,0.000000
50%,273.000000,396.000000,6.355677e+06,30.000000,197.360000,26.000000,196.350000,0.000000
75%,338.000000,637.000000,8.627548e+06,32.000000,2598.415000,236.000000,2575.520000,0.000000
max,559.000000,999.000000,1.217584e+07,6968.000000,392036.460000,434200.000000,392036.460000,77796.690000


#**3.Transformación y Preprocesamiento - Data Wrangling**

In [ ]:
df.dtypes

In [ ]:
df.columns

In [ ]:
df.columns = df.columns.str.lower().str.replace(r"\W+", "", regex=True)

df.columns


In [ ]:
df['tds'] = df['tds'].fillna(0).astype(int)
df['days'] = df['days'].fillna(0).astype(int)
df['servicestartdate'] = pd.to_datetime(df['servicestartdate'], format='%m/%d/%Y')
df['serviceenddate'] = pd.to_datetime(df['serviceenddate'], format='%m/%d/%Y')
df['revenuemonth'] = pd.to_datetime(df['revenuemonth'], format='%Y-%m')

df.dtypes

In [ ]:
df.isnull().sum()

In [ ]:
diccionario_datos = ({"Nombre de columna": df.columns,
                    "Tipo de dato": [str(df[col].dtype) for col in df.columns]})

print(diccionario_datos)

# **4. Análisis Descriptivo de Datos**

## - Preguntas a responder con el AD
## - Explicación de Estadísticas Básicas Aplicadas, Visualización de Datos con Gráficos, y Resumen de las respuestas

# **5. Análisis Exploratorio de los Datos**

## - Preguntas a responder
## - Estadísticas Empleadas, visualización y Resumen de las respuestas encontradas.

# **6. Análisis Predictivo**

## - Preprocesamiento para Análisis Predictivo: Problemas, Técnicas y Código

## - Modelado Predictivo
### - Modelo elegido y parámetros
### - Entrenamiento y Evaluación
## - Resultados


# **7. Conclusiones y Trabajos Futuros**

## - Resumen del trabajo realizado, hallazgos y posibles mejoras